In [1]:
import requests
from bs4 import BeautifulSoup
import sys
import pandas as pd
import time,random
import json
import datetime
headers = {
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'Accept-Encoding': 'gzip, deflate',
'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
'Connection': 'keep-alive',
'Cookie': 'ipmsperf_uuid=-3462900230048563768; ccsession=202105241319040000374b374b91b5; _KONEPS_PAGE=%3C%21%23%25%3EtbidListUrl%3C%40%25%7E%3E%2Fep%2Ftbid%2FtbidList.do%3Farea%3D%26bidNm%3D%26bidSearchType%3D1%26fromBidDt%3D2021%252F05%252F24%26fromOpenBidDt%3D%26instNm%3D%26radOrgan%3D1%26regYn%3DY%26searchDtType%3D1%26searchType%3D1%26taskClCds%3D%26toBidDt%3D2021%252F05%252F24%26toOpenBidDt%3D; JSESSIONID=5WPTgrpLSHXbn9mLQ6LGkQvFB50rkhFyLzXxpTyy8pJHQG1LHxMp!1373003672',
'Host': 'www.g2b.go.kr:8101',
'Referer': 'http://www.g2b.go.kr/pt/menu/frameTgong.do?url=http://www.g2b.go.kr:8101/ep/tbid/tbidList.do?taskClCds=&bidNm=&searchDtType=1&fromBidDt=2021/04/24&toBidDt=2021/05/24&fromOpenBidDt=&toOpenBidDt=&radOrgan=1&instNm=&area=&regYn=Y&bidSearchType=1&searchType=1',
'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.272 Whale/2.9.117.22 Safari/537.36',
}



In [2]:
###startdate/end_date 뽑기
#from datetime import date, timedelta
import datetime
monday_list=[]
start_end_list = []
#yearmonday_list=[[2017,1,2],[2018,1,1],[2019,1,7],[2020,1,6],[2021,1,4]]
yearmonday_list=[[2020,8,3],[2021,1,4],[2022,1,3],[2023,1,2],[2024,1,1]]
for yearmon in yearmonday_list:
    year = yearmon[0]
    date_object = datetime.date(year, yearmon[1], yearmon[2])
    date_object += datetime.timedelta(days=1-date_object.isoweekday())
    
    while date_object.year == year:
        #print(date_object.strftime("%Y%m%d"))
        monday_list.append(date_object.strftime("%Y%m%d"))
        date_object += datetime.timedelta(days=7)
        
for date_ in monday_list : 
    #print(date)
    w_date = datetime.datetime.now().strftime("%Y%m%d")
    return_start_date_tmp = datetime.datetime.strptime(date_, '%Y%m%d') ###수정
    return_start_date=datetime.datetime.strftime(return_start_date_tmp,'%Y%m%d')
    return_end_date = return_start_date_tmp + datetime.timedelta(days=6)
    return_end_date=datetime.datetime.strftime(return_end_date,'%Y%m%d')
    date_scale = return_start_date+"_scale"
    if return_end_date < w_date:
        start_end_list.append((return_start_date,return_end_date))
    else : pass
    #print(return_start_date,return_end_date)stand_scale
len(start_end_list[:])

47

In [3]:
last_mon_day = start_end_list[-1][0]
last_sun_day = start_end_list[-1][1]

last_mon_y =  last_mon_day[:4]
last_sun_y =  last_sun_day[:4]
last_mon_m =  last_mon_day[4:6]
last_sun_m =  last_sun_day[4:6]
last_mon_d =  last_mon_day[6:]
last_sun_d =  last_sun_day[6:]
print(last_mon_y,last_mon_m,last_mon_d )
print(last_sun_y,last_sun_m,last_sun_d )

2021 06 21
2021 06 27


In [4]:

page_num = 1
corsor =True
datas=[]
final_num = 0





while corsor :
    #print(page_num)
    url =f'http://www.g2b.go.kr:8101/ep/tbid/tbidList.do?area=&bidNm=&bidSearchType=1&currentPageNo={page_num}&fromBidDt={last_mon_y}%2F{last_mon_m}%2F{last_mon_d}&fromOpenBidDt=&instNm=&maxPageViewNoByWshan={page_num}&radOrgan=1&regYn=Y&searchDtType=1&searchType=1&taskClCds=&toBidDt={last_sun_y}%2F{last_sun_m}%2F{last_sun_d}&toOpenBidDt=&'
    sys.stdout.write(f'\r{url}')
    sys.stdout.flush()
    res = requests.get(url, headers= headers)
    if res.status_code ==200 :
        soup = BeautifulSoup(res.text, 'html.parser')
        item_list = soup.find('tbody').find_all('tr')
        #item_list[0]
       
        for item in item_list : 
            tmp_dic={}
            tl_list = item.find_all('td', class_='tl')
            td_list = item.find_all('td')
            #print(tl_list)
            if len(tl_list) == 4 :
                tmp_dic['cate_nm'] = td_list[0].text
                tmp_dic['content_cd']= td_list[1].text
                tmp_dic['cate_nm2']= td_list[2].text
                tmp_dic['title_nm'] = tl_list[0].text
                tmp_dic['gover_nm'] = tl_list[1].text +'/' + tl_list[2].text
                tmp_dic['method'] = tl_list[3].text
                tmp_dic['due_date'] = item.find('td', class_='tc').text
                tmp_dic['url'] = item.find('a')['href']
                #datas.append(tmp_dic)
                js = json.dumps(tmp_dic)
                with open(f'./result/narajanteo_{last_mon_day}_{last_sun_day}.jl','a') as f:
                    f.write(js+'\n')
                
            else :
                print('제외',page_num)
                final_num +=1
        
        page_num+=1
        if final_num ==5 :
            corsor = False
    else : corsor = False
    time.sleep(1)
#df = pd.DataFrame(datas)


http://www.g2b.go.kr:8101/ep/tbid/tbidList.do?area=&bidNm=&bidSearchType=1&currentPageNo=323&fromBidDt=2021%2F06%2F21&fromOpenBidDt=&instNm=&maxPageViewNoByWshan=323&radOrgan=1&regYn=Y&searchDtType=1&searchType=1&taskClCds=&toBidDt=2021%2F06%2F27&toOpenBidDt=&제외 323
http://www.g2b.go.kr:8101/ep/tbid/tbidList.do?area=&bidNm=&bidSearchType=1&currentPageNo=993&fromBidDt=2021%2F06%2F21&fromOpenBidDt=&instNm=&maxPageViewNoByWshan=993&radOrgan=1&regYn=Y&searchDtType=1&searchType=1&taskClCds=&toBidDt=2021%2F06%2F27&toOpenBidDt=&제외 993
http://www.g2b.go.kr:8101/ep/tbid/tbidList.do?area=&bidNm=&bidSearchType=1&currentPageNo=994&fromBidDt=2021%2F06%2F21&fromOpenBidDt=&instNm=&maxPageViewNoByWshan=994&radOrgan=1&regYn=Y&searchDtType=1&searchType=1&taskClCds=&toBidDt=2021%2F06%2F27&toOpenBidDt=&제외 994
http://www.g2b.go.kr:8101/ep/tbid/tbidList.do?area=&bidNm=&bidSearchType=1&currentPageNo=995&fromBidDt=2021%2F06%2F21&fromOpenBidDt=&instNm=&maxPageViewNoByWshan=995&radOrgan=1&regYn=Y&searchDtType=1

In [6]:
df=pd.read_json(f'./result/narajanteo_{last_mon_day}_{last_sun_day}.jl',lines=True)
df.to_excel(f'./result/narajanteo_{last_mon_day}_{last_sun_day}.xlsx',encoding='cp949',index= False)

In [ ]:
df

In [27]:
df.to_excel('narajangteo_20210610_20210617.xlsx',encoding='cp949')